In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql
#결과 여러줄 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

## 제품을 크롤링 해서 DB에 저장하기
1. DB 서버 연결
2. DB 및 테이블 생성
3. 데이터 크롤링
   - pandas를 활용해서 테이블 형식에 맞게 정제
4. 데이터 insert
5. commit으로 db에 반영
6. db close

In [4]:
def conn(d_name):
  import pymysql
  host_name='localhost'
  host_port=3306
  username='root'
  passwd='1234'
  database_name = d_name
  db = pymysql.connect(host=host_name, 
												port=host_port, 
												user=username, 
												passwd=passwd, 
												db=database_name, 
												charset='utf8')
  return db

In [ ]:
#beauty_shop 데이터 베이스 만들기
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234')
cursor=db.cursor()
sql = 'CREATE DATABASE beauty_shop'
cursor.execute(sql)

In [68]:
#데이터베이스 접근
db = conn('beauty_shop')
cursor = db.cursor()

In [53]:
#데이터베이스 목록 확인
sql = 'show databases'
cursor.execute(sql)
result = cursor.fetchall()
result

7

(('beauty_shop',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',),
 ('world',))

In [55]:
#beauty_shop 사용 선택
cursor.execute('use beauty_shop')

0

In [59]:
sql = 'desc product'
cursor.execute(sql)

6

In [ ]:
#테이블 drop
sql = 'drop table product'
cursor.execute(sql)

0

In [70]:
#product 테이블 생성
sql = '''
		CREATE TABLE product (
				PRODUCT_CODE int AUTO_INCREMENT NOT NULL,
				PRODUCT_NAME VARCHAR(200) NOT NULL,
				PRODUCT_PRICE VARCHAR(100),
				DISCOUNT_PRICE VARCHAR(100),
        DISCOUNT_RATE VARCHAR(100),
				PRODUCT_LINK VARCHAR(200),
				PRIMARY KEY(PRODUCT_CODE)
		);
'''

cursor.execute(sql)
db.commit()

0

In [9]:
sql = "show tables"
cursor.execute(sql)
result = cursor.fetchall()
result

1

(('product',),)

In [10]:
sql = "desc product"
cursor.execute(sql)
result = cursor.fetchall()
result

6

(('PRODUCT_CODE', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('PRODUCT_NAME', 'varchar(200)', 'NO', '', None, ''),
 ('PRODUCT_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('DISCOUNT_PRICE', 'varchar(100)', 'YES', '', None, ''),
 ('DISCOUNT_RATE', 'varchar(100)', 'YES', '', None, ''),
 ('PRODUCT_LINK', 'varchar(200)', 'YES', '', None, ''))

In [43]:
import requests

#url = "http://jolse.com/category/toners-mists/1019"
url='https://jolse.com/category/makeup/4722'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req= requests.get(url, headers=header)
html = req.text
soup = BeautifulSoup(html, 'lxml')

In [ ]:
#전체 상품 선택자
products = soup.select('div.xans-element-.xans-product.xans-product-listnormal.ec-base-product > ul > li')
products

#잘 가지고 오는지 테스트

product_name = products.select_one('strong > a > span:nth-child(2)').get_text()
product_name

baseurl = 'https://jolse.com'
product_url = products.select_one('strong > a').get('href')
product_url = baseurl + product_url
product_url

product_price = products.select_one('li.xans-record- > span').get_text()
product_price


discounted_price = products.select_one('li.xans-record- +li > span').get_text().split(' ')[1]
discounted_price

discounted_rate = products.select_one('li.xans-record- +li > span > span').get_text().strip('()')
discounted_rate

In [ ]:
#전체 상품 선택자
products = soup.select('div.xans-element-.xans-product.xans-product-listnormal.ec-base-product > ul > li')

for n,product in enumerate(products):
  product_name = product.select_one('strong > a > span~span').get_text()
  print(f'{n}번째 상품명 = {product_name}')

# for n,product in enumerate(products, start=1):
#   baseurl = 'https://jolse.com'
#   product_url = product.select_one('strong > a').get('href')
#   product_url = baseurl + product_url
#   print(f'{n}번째 제품 링크 = {product_url}')

# baseurl = 'https://jolse.com'
# product_url = products.select_one('strong > a').get('href')
# product_url = baseurl + product_url
# product_url

# product_price = products.select_one('li.xans-record- > span').get_text()
# product_price


# discounted_price = products.select_one('li.xans-record- +li > span').get_text().split(' ')[1]
# discounted_price

# discounted_rate = products.select_one('li.xans-record- +li > span > span').get_text().strip('()')
# discounted_rate

0번째 상품명 = mude inspire skinny curling mascara 3ml
1번째 상품명 = CORALHAZE Glow Lock Jelly Tint 4g
2번째 상품명 = A'PIEU Honey & Milk Lip Oil 5g
3번째 상품명 = Dinto Blur-Glowy Lip Tint 3.5g (30 colors)
4번째 상품명 = SKINFOOD SALMON DARKCIRCLE CONCEALER CREAM 10g
5번째 상품명 = 2aN Gleaming Tension Pact 13g (cushion foundation)
6번째 상품명 = MISSHA M Perfect Cover BB Cream 50ml
7번째 상품명 = Unleashia Non-Sticky Dazzle Tint 4g
8번째 상품명 = Unleashia Shaper Pomade Eyebrow Fixer 8g #NO.1 CLEAR
9번째 상품명 = ETUDE Color Lasting Glow Stick 3.2g
10번째 상품명 = Peripera Heart Jam Glow Lip
11번째 상품명 = MISSHA 4D Mascara 7g
12번째 상품명 = SKINFOOD PEACH COTTON PORE BLUR PACT 4g
13번째 상품명 = BBIA Last Auto Gel Eyeliner 0.3g (22AD)
14번째 상품명 = 2aN Pure Glash Highlighter 2.7g
15번째 상품명 = BBIA GLOW LIP TINT 3.2g
16번째 상품명 = ETUDE Drawing Eye Brow 0.25g
17번째 상품명 = ETUDE My Own Gemstone Look at My Eyes 2g
18번째 상품명 = 2aN Dew Glowy Tint 3.4g
19번째 상품명 = ETUDE Fixing Tint 4g
20번째 상품명 = CLIO KILL COVER MESH GLOW CUSHION Set 15g + REFILL 15g
21번째 상품명 = Unlea

In [ ]:

def get_product_info(product):
	try:
		product_name = product.select_one('strong > a > span:nth-child(2)').get_text()
	except Exception:
		product_name = ' '
	
	base_url = 'https://jolse.com'
	detail_url = product.select_one('strong > a').get('href')
	product_url = base_url+detail_url

	product_price = product.select_one('li.xans-record- > span').get_text().split(' ')[1]
  

	try: 
		discounted_price = product.select_one('li.xans-record- +li > span').get_text().split(' ')[1]
	except Exception:
		discounted_price = 0.0

	try:
		discounted_rate = product.select_one('li.xans-record- +li > span > span').get_text().strip('()')
	except Exception:
		discounted_rate = 0.0
      
	
	return {'product_name':product_name,'product_url':product_url,
					'product_price':product_price, 'discounted_price':discounted_price,
					'discounted_rate':discounted_rate }


데이터 베이스 컬럼
('PRODUCT_CODE', 'int', 'NO', 'PRI', None, 'auto_increment'),
('PRODUCT_NAME', 'varchar(200)', 'NO', '', None, ''),
('PRODUCT_PRICE', 'varchar(100)', 'YES', '', None, ''),
('DISCOUNT_PRICE', 'varchar(100)', 'YES', '', None, ''),
('DISCOUNT_RATE', 'varchar(100)', 'YES', '', None, ''),
('PRODUCT_LINK', 'varchar(200)', 'YES', '', None, '')

In [46]:
def data_save(product_info):
  
	sql = """INSERT INTO product 
	(PRODUCT_NAME, PRODUCT_PRICE, DISCOUNT_PRICE, DISCOUNT_RATE, PRODUCT_LINK) 
	VALUES (%s, %s, %s, %s, %s)"""

	values = (
	product_info['product_name'],
	product_info['product_price'],
	product_info['discounted_price'],
	product_info['discounted_rate'],
	product_info['product_url']
	)

	try:
		cursor.execute(sql, values)
		db.commit()
	except Exception as e:
		print(e)

In [47]:
def get_page_products(url):
  header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
  req= requests.get(url, headers=header)
  html = req.text
  soup = BeautifulSoup(html, 'lxml')
  #제품 정보 담고있는 부분
  products = soup.select('div.xans-element-.xans-product.xans-product-listnormal.ec-base-product > ul > li')
  
  for product in products:
      product_info = get_product_info(product)
      data_save(product_info)

In [71]:
from tqdm import tqdm
import time
# c_url = 'https://jolse.com/category/toners-mists/1019?page='
c_rul_m = 'https://jolse.com/category/makeup/4722?page='

last = 10
for i in tqdm(range(1,last+1)):
  last_url=c_rul_m+str(i)
  get_page_products(last_url)

 30%|███       | 3/10 [00:02<00:06,  1.17it/s]

(1366, "Incorrect string value: '\\xF0\\x9F\\x8C\\xB7[1...' for column 'PRODUCT_NAME' at row 1")


100%|██████████| 10/10 [00:08<00:00,  1.13it/s]


In [ ]:
db.commit()

In [61]:
sql = "select * from product"
cursor.execute(sql)
result = cursor.fetchall()
result

399

((1,
  'mude inspire skinny curling mascara 3ml',
  '14.00',
  '7.29',
  '48% OFF',
  '/product/mude-inspire-skinny-curling-mascara-3ml/69879/category/4722/display/1/https://jolse.com'),
 (2,
  'CORALHAZE Glow Lock Jelly Tint 4g',
  '19.00',
  '6.79',
  '64% OFF',
  '/product/coralhaze-glow-lock-jelly-tint-4g/83183/category/4722/display/1/https://jolse.com'),
 (3,
  "A'PIEU Honey & Milk Lip Oil 5g",
  '11.00',
  '3.79',
  '66% OFF',
  '/product/apieu-honey-milk-lip-oil-5g/13087/category/4722/display/1/https://jolse.com'),
 (4,
  'Dinto Blur-Glowy Lip Tint 3.5g (30 colors)',
  '20.00',
  '6.99',
  '65% OFF',
  '/product/dinto-blur-glowy-lip-tint-35g-30-colors/64429/category/4722/display/1/https://jolse.com'),
 (5,
  'SKINFOOD SALMON DARKCIRCLE CONCEALER CREAM 10g',
  '11.50',
  '2.19',
  '81% OFF',
  '/product/skinfood-salmon-darkcircle-concealer-cream-10g/79390/category/4722/display/1/https://jolse.com'),
 (6,
  '2aN Gleaming Tension Pact 13g (cushion foundation)',
  '33.00',
  '11.99'

In [67]:
db.close()

In [72]:
db = conn('beauty_shop')
sql = 'select * from product'
df = pd.read_sql(sql,db)
df

C:\Users\hhhmh\AppData\Local\Temp\ipykernel_23760\2610791739.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql,db)


,PRODUCT_CODE,PRODUCT_NAME,PRODUCT_PRICE,DISCOUNT_PRICE,DISCOUNT_RATE,PRODUCT_LINK
0,1,mude inspire skinny curling mascara 3ml,14.00,7.29,48% OFF,/product/mude-inspire-skinny-curling-mascara-3...
1,2,CORALHAZE Glow Lock Jelly Tint 4g,19.00,6.79,64% OFF,/product/coralhaze-glow-lock-jelly-tint-4g/831...
2,3,A'PIEU Honey & Milk Lip Oil 5g,11.00,3.79,66% OFF,/product/apieu-honey-milk-lip-oil-5g/13087/cat...
3,4,Dinto Blur-Glowy Lip Tint 3.5g (30 colors),20.00,6.99,65% OFF,/product/dinto-blur-glowy-lip-tint-35g-30-colo...
4,5,SKINFOOD SALMON DARKCIRCLE CONCEALER CREAM 10g,11.50,2.19,81% OFF,/product/skinfood-salmon-darkcircle-concealer-...
...,...,...,...,...,...,...
394,395,FRESHIAN Egg-like Cushion Vegan 12g,35.00,29.75,15% OFF,/product/freshian-egg-like-cushion-vegan-12g/7...
395,396,FRESHIAN Egg-like Tone up Cushion (Refill) 12g...,23.00,19.55,15% OFF,/product/freshian-egg-like-tone-up-cushion-ref...
396,397,FRESHIAN Egg-like Tone up Cushion 12g #Nudy Pink,35.00,29.75,15% OFF,/product/freshian-egg-like-tone-up-cushion-12g...
397,398,FRESHIAN Vegan Serum Lip Tint 6g,26.00,22.10,15% OFF,/product/freshian-vegan-serum-lip-tint-6g/7129...
